In [16]:
import re
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os 
import shutil
import hashlib
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## Top 500 文章爬蟲

In [2]:
data = {}
count = 1
browser = webdriver.Chrome('../chromedriver.exe')
url = 'https://www.cupoy.com/newsfeed/topstory'
browser.get(url)
time.sleep(10)
while True:
    
    # 基礎爬蟲
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    res = soup.find_all('div', class_='sc-eEieub sc-iuDHTM ibJqYc')
    # 迴圈爬取資料並分類
    for item in res:
        data.setdefault(count,{})
        #分類或來源
        data[count]['category'] = item.find('div',{'class':'sc-gacfCG bPSpUf'}).string
        #文章來源網址
        data[count]['link'] = item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('href')
        #標題
        data[count]['title'] = item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('title')
        #簡介
        data[count]['summary'] = item.find('p',{'class':'sc-FQuPU sc-ciodno bvnzOw'}).string
        
        #來源網址(若非聯外網站則自動輸入為cupoy網頁)
        source_data = re.findall('[\S]+[com|com.tw]+',item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('href'))
        if len(source_data)>0:
            data[count]['source'] = source_data[0]
        else:
            data[count]['source'] = 'https://www.cupoy.com'
        #文章標籤序列    
        count += 1
        
    #爬蟲進度顯示
    print(f"update... {count}")
    
    
    # 模擬器頁面下滑
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

    # 計算新頁面高度
    browser.execute_script("return document.body.scrollHeight")
    
    # 設置文章數
    if count > 500:
        print("collected over")
        break
    time.sleep(5) 
    
browser.quit()
data

update... 16
update... 33
update... 49
update... 66
update... 83
update... 99
update... 115
update... 132
update... 149
update... 166
update... 183
update... 200
update... 217
update... 234
update... 251
update... 267
update... 284
update... 301
update... 318
update... 335
update... 352
update... 369
update... 386
update... 403
update... 419
update... 436
update... 453
update... 470
update... 487
update... 504
collected over


{1: {'category': '科技',
  'link': 'https://technews.tw/2020/04/19/olive-oil-industry-under-increasing-threat-from-olive-leprosy/?utm_source=fb_tn&utm_medium=facebook',
  'title': '橄欖樹瘟疫大流行，傳遍南歐三大產區的「橄欖樹痲瘋病」 | TechNews 科技新報',
  'summary': '正當人類為 COVID-19（武漢肺炎）疫情而急破頭時，地中海地區的橄欖樹們也跟著傳出「疫情」。\n橄欖樹生病了\n近日，人類世界的 COVID-19（武漢肺炎）疫情讓世界各國忙得團團轉，但是南歐各地的橄欖樹，現在也傳出傳染病盛行的消息。一種被《衛報》比喻為「橄欖樹痲瘋病」的植物疾病，在各大橄欖油產...',
  'source': 'https://technews.tw/2020/04/19/olive-oil-industry-under-increasing-threat-from-olive-leprosy/?utm_source=fb_tn&utm_medium=faceboo'},
 2: {'category': '科技',
  'link': 'https://technews.tw/2020/04/19/trade-coffee-delivery/?utm_source=fb_tn&utm_medium=facebook',
  'title': '無法出門好悶，美國小店選物盒雙贏出招 | TechNews 科技新報',
  'summary': '平時，商家間常處於競爭狀態，但當彼此較勁的商家，全都因為疫情必須暫時關閉時，該怎麼一起度過難關？大家都活下來，成為比競爭更重要的事。\n\n\n\n近期，不僅臺灣商界開始跨界聯盟，國外的店家也用無比創意來共同對抗疫情，讓客戶不出門，也能感受到商家的體貼服務。例如，把不同商家的商品，放在同一個盒子裡販...',
  'source': 'https://technews.tw/2020/04/19/trade-coffee-delivery/?utm_source=fb_tn&utm_medium=faceboo'},
 3: {'ca

## 資料整理

In [3]:
dataframe = pd.DataFrame(data)
df = dataframe.T
df.head()

,category,link,title,summary,source
1,科技,https://technews.tw/2020/04/19/olive-oil-indus...,橄欖樹瘟疫大流行，傳遍南歐三大產區的「橄欖樹痲瘋病」 | TechNews 科技新報,正當人類為 COVID-19（武漢肺炎）疫情而急破頭時，地中海地區的橄欖樹們也跟著傳出「疫情...,https://technews.tw/2020/04/19/olive-oil-indus...
2,科技,https://technews.tw/2020/04/19/trade-coffee-de...,無法出門好悶，美國小店選物盒雙贏出招 | TechNews 科技新報,平時，商家間常處於競爭狀態，但當彼此較勁的商家，全都因為疫情必須暫時關閉時，該怎麼一起度過難...,https://technews.tw/2020/04/19/trade-coffee-de...
3,科技,https://technews.tw/2020/04/19/nasas-curiosity...,NASA 分享員工在家上班情況，如何在家中遙控火星探測車「好奇號」 | TechNews 科技新報,受武漢肺炎疫情影響，美國太空總署 NASA 日前也宣佈要員工留在家中工作，希望維持日常的外太...,https://technews.tw/2020/04/19/nasas-curiosity...
4,Engadget,https://chinese.engadget.com/chinese-2020-04-1...,LG 官方公佈新機 Velvet 外觀影片,LG 最近頻頻預熱的新手機 Velvet，今天又迎來了一次官方的全方位曝光。在已經確認的雙曲...,https://chinese.engadget.com/chinese-2020-04-1...
5,Engadget,https://chinese.engadget.com/chinese-2020-04-1...,微軟：Windows 10 五月更新的最後一版預覽已經就緒,在開始正式放緩 Windows 10 的更新節奏前，微軟近日端出了五月更新的最後一版預覽。I...,https://chinese.engadget.com/chinese-2020-04-1...


Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            494, 495, 496, 497, 498, 499, 500, 501, 502, 503],
           dtype='int64', length=503)

In [17]:
def create_folder(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
        
def hash_str(string):
    return hashlib.sha1(string.encode('utf-8')).hexdigest()

In [20]:
path = './temp'
contents = []
count = 1
tmp_path = "./tmp"
create_folder(tmp_path)

for index, row in tqdm(df.iterrows(), total=len(df)):
    # check url correct
    r = requests.get(row.link)
    if r.status_code != requests.codes.ok: continue
    
    # extract text from website
    text = str()
    soup = BeautifulSoup(r.text, "html5lib")
    for tag in soup.find_all('p'):
        text += tag.text   
        
    # save as .txt
    ## use hash as file name to avoid filename error
    with open(os.path.join(tmp_path, hash_str(row.title) + ".txt"), 'w', encoding='utf-8') as f:
        f.write(text)
    
    #break

  0%|                                                                                          | 0/503 [00:00<?, ?it/s]

Successfully created the directory ./tmp 


 94%|██████████████████████████████████████████████████████████████████████████▉     | 471/503 [08:43<01:39,  3.11s/it]

SSLError: ("read error: Error([('SSL routines', 'ssl3_read_n', 'unexpected eof while reading')])",)

In [26]:
import jieba
import jieba.analyse

jieba.analyse.set_stop_words("./jieba_TW_stopWord.txt")

Exception: jieba: file does not exist: C:\Users\ASROCK\Documents\GitHub\2nd-PyCrawlerMarathon\Day_038-040\jieba_TW_stopWord.txt

In [27]:
keyword_list = []
for txt_name in tqdm(os.listdir(tmp_path)):
    txt_path = os.path.join(tmp_path, txt_name)
    with open(txt_path, 'r', encoding='utf-8') as f:
        text = str()
        for line in f:
            text += line.strip() + ' '
        
        # check if .txt empty
        if len(text) == 0: continue 
            
        # extract only top-50 keywords from each website
        keyword_list += jieba.analyse.extract_tags(text, 
                                                   topK=50, 
                                                   withWeight=False, 
                                                   allowPOS=())


  0%|                                                                                          | 0/361 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ASROCK\AppData\Local\Temp\jieba.cache
Loading model cost 0.698 seconds.
Prefix dict has been built successfully.

100%|████████████████████████████████████████████████████████████████████████████████| 361/361 [00:03<00:00, 93.41it/s]


In [28]:
def count_segment_freq(keyword_list):
    keyword_df = pd.DataFrame(keyword_list, columns=['keyword'])
    keyword_df['count'] = 1
    keyword_freq = keyword_df.groupby('keyword')['count'].sum().sort_values(ascending=False)
    keyword_freq = pd.DataFrame(keyword_freq)
    return keyword_freq

keyword_freq = count_segment_freq(keyword_list)
keyword_freq.head(20)

,count
keyword,
我們,120
2020,85
一個,82
因為,66
台灣,59
時間,58
19,56
疫情,51
這個,50
